In [3]:
install.packages('tidyverse')
install.packages('magrittr')
install.packages('lubridate')
install.packages('iNEXT')
install.packages('vegan')
install.packages('networkD3')
install.packages('permute')
install.packages('dplyr')
install.packages('tidyr')
install.packages("rlang")

also installing the dependencies ‘haven’, ‘readr’


Warning message in install.packages("tidyverse"):
“installation of package ‘readr’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘haven’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘tidyverse’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in install.packages("networkD3"):
“installation of package ‘networkD3’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML inde

In [4]:
library('tidyverse')
library('magrittr')
library('lubridate')
library('iNEXT')
library('vegan')
library('networkD3')
library('permute')
library('tidyr')
library('dplyr')

ERROR: Error in library("tidyverse"): there is no package called ‘tidyverse’


In [5]:
sessionInfo()

R version 4.1.0 (2021-05-18)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 21.10

Matrix products: default
BLAS/LAPACK: /home/grati/miniconda3/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=pt_BR.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=pt_BR.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=pt_BR.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=pt_BR.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=pt_BR.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] fansi_0.4.2       digest_0.6.27     utf8_1.2.2        crayon_1.4.2     
 [5] IRdisplay_1.0     repr_1.1.3        lifecycle_1.0.1   jsonlite_1.7.2   
 [9] evaluate_0.14     pillar_1.6.4      rlang_0.4.12      uuid_0.1-4       
[13] vctrs_0.3.8       ellipsis_0.3.2    IRkernel_1.2      tools_4.1.0    

In [6]:
# lendo o arquivo de entrada

df <- read_csv("../data/data_pcrs.csv")

df <- df %>% pivot_longer(names_to = "Localidade", values_to = "Abundância")

distinct(df, Localidade)

ERROR: Error in read_csv("../data/data_pcrs.csv"): could not find function "read_csv"


In [ ]:
# Sankey

nodes <- data.frame(name = c(as.character(df$Espécie), as.character(df$Localidade)) %>% unique())


df$IDsource <- match(df$Espécie, nodes$name) - 1
df$IDtarget <- match(df$Localidade, nodes$name) - 1

ColourScal <- 'd3.scaleOrdinal() .range(["#FDE725FF","#B4DE2CFF","#6DCD59FF","#35B779FF",
"#1F9E89FF","#26828EFF","#31688EFF","#3E4A89FF","#482878FF","#440154FF"])'

sankeyNetwork(
  Links = df, Nodes = nodes,
  Source = "IDsource", Target = "IDtarget",
  Value = "Abundância", NodeID = "name",
  sinksRight = FALSE, colourScale = ColourScal, nodeWidth = 40, fontSize = 13, nodePadding = 20
)

In [ ]:
# top 5 specie

top_specie <- df %>% arrange(-Abundância) %>% slice(1:5) 

In [ ]:
# NMDS

rich <- df  %>%  group_by(Localidade) %>% 
  filter(Abundância > 0) %>%
  summarise(rich = n_distinct(Espécie))

data_nmds <- df %>%
  group_by(Espécie, Localidade) %>%
  pivot_wider(names_from = Espécie, values_from = Abundância) %>%
  mutate(
    across(everything(), ~ replace_na(.x, 0))
  ) %>% left_join(rich, by = "Localidade")


run_nmds <- data_nmds

run_nmds$Localidade <- NULL
run_nmds$rich <- NULL

dist_bray <- vegdist(run_nmds, method = "bray", binary = TRUE)

dist_bray

nmds <- metaMDS(dist_bray)

scores(nmds)  %>%
  as_tibble() %>%
  cbind(data_nmds) %>%
  as_tibble()%>%
  ggplot(aes(x = NMDS1, y = NMDS2)) +
  geom_point(aes(size = rich)) +
  stat_ellipse(geom = "polygon", aes(group = rich), alpha = 0.3) +
  annotate("text", x = -2, y = 0.95, label = paste0("stress: ", format(nmds$stress, digits = 4)), hjust = 0) +
  theme_bw()


In [ ]:
# Permanova

adonis(dist_bray~data_nmds$Localidade, permutations = 1000)

In [ ]:
# Rich x Abund

rich_abund <- data_nmds %>% 
  pivot_longer(!c(Localidade, rich), names_to = "Espécie", values_to = "abund") %>% 
  filter(Espécie %in% top_specie$Espécie)

ggplot(rich_abund, aes(x = rich, y = abund)) +
  geom_point() +
  geom_smooth(method = "lm", alpha = .15) +
  facet_wrap(~Espécie)


In [ ]:
# Diversidade

abund <- df %>%
  pivot_wider(names_from = Localidade, values_from = Abundância)  %>%
    column_to_rownames(var = "Espécie")

resultados_tabanidae <- iNEXT(abund,
                              q = c(0, 1, 2),
                              datatype = "abundance",
                              endpoint = 400
)

In [ ]:
# Resultado 0 = Species richness; 1 = Shannon diversity; 2 = Simpson diversity

resultados_tabanidae$AsyEst

ggiNEXT(resultados_tabanidae, type = 1, facet.var = 'site') + theme_light() + 
  facet_wrap(~site, ncol=3, scales = 'free') 
